In [1]:
############### Product Report
# Importing Necessary libraries
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
from sqlalchemy import create_engine

server = 'LAPTOP-LFBT0G3K'
database = 'DataWarehouse'
driver = '{ODBC Driver 18 for SQL Server}'

In [8]:
try:
    quoted_driver = urllib.parse.quote_plus(driver)
    connection_uri = (
        f'mssql+pyodbc:///?odbc_connect='
        f'DRIVER={quoted_driver};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'Trusted_Connection=yes;'
        f'Encrypt=yes;'
        f'TrustServerCertificate=yes;'
    )
    sql_analysis_query="""
                        WITH base_query AS (
                        --1) Base Query: Retrieves core columns from fact_sales and dim_products
                            SELECT
                                f.order_number,
                                f.order_date,
                                f.customer_key,
                                f.sales_amount,
                                f.quantity,
                                p.product_key,
                                p.product_name,
                                p.category,
                                p.subcategory,
                                p.cost
                            FROM gold.fact_sales f
                            LEFT JOIN gold.dim_products p
                                ON f.product_key = p.product_key
                            WHERE order_date IS NOT NULL
                        ),

                        product_aggregations AS (
                        --2) Product Aggregations: Summarizes key metrics at the product level
                        SELECT
                            product_key,
                            product_name,
                            category,
                            subcategory,
                            cost,
                            DATEDIFF(MONTH, MIN(order_date), MAX(order_date)) AS lifespan,
                            MAX(order_date) AS last_sale_date,
                            COUNT(DISTINCT order_number) AS total_orders,
                            COUNT(DISTINCT customer_key) AS total_customers,
                            SUM(sales_amount) AS total_sales,
                            SUM(quantity) AS total_quantity,
                            ROUND(AVG(CAST(sales_amount AS FLOAT) / NULLIF(quantity, 0)),1) AS avg_selling_price

                        FROM base_query
                        GROUP BY
                            product_key,
                            product_name,
                            category,
                            subcategory,
                            cost
                        )

                        --3) Final Query: Combines all product results into one output
                        SELECT
                            product_key,
                            product_name,
                            category,
                            subcategory,
                            cost,
                            last_sale_date,
                            DATEDIFF(MONTH, last_sale_date, GETDATE()) AS recency_in_months,
                            
                            CASE
                                WHEN total_sales > 50000 THEN 'High-Performer'
                                WHEN total_sales >= 10000 THEN 'Mid-Range'
                                ELSE 'Low-Performer'
                            END AS product_segment,

                            lifespan,
                            total_orders,
                            total_sales,
                            total_quantity,
                            total_customers,
                            avg_selling_price,
                            
                            -- Average Order Revenue (AOR)
                            CASE
                                WHEN total_orders = 0 THEN 0
                                ELSE total_sales / total_orders
                            END AS avg_order_revenue,

                            -- Average Monthly Revenue
                            CASE
                                WHEN lifespan = 0 THEN total_sales
                                ELSE total_sales / lifespan
                            END AS avg_monthly_revenue
                        FROM product_aggregations;
    """
    print(f"Attempting to connect to {server}/{database} using driver: {driver}")
    engine = create_engine(connection_uri)
    print("Successfully connected to SQL Server!")
    
    df_product_report=pd.read_sql(sql_analysis_query,engine)


except Exception as e:
    print(f"An unexpected error occurred: {e}")

Attempting to connect to LAPTOP-LFBT0G3K/DataWarehouse using driver: {ODBC Driver 18 for SQL Server}
Successfully connected to SQL Server!


In [9]:
# Test for Data
print('first 5 rows fo sales analysis Data : ')
print(df_product_report.head())
print('\nSales Analysis Data info : ')
print(df_product_report.info())

first 5 rows fo sales analysis Data : 
   product_key             product_name category     subcategory  cost  \
0            3   Mountain-100 Black- 38    Bikes  Mountain Bikes  1898   
1            4   Mountain-100 Black- 42    Bikes  Mountain Bikes  1898   
2            5   Mountain-100 Black- 44    Bikes  Mountain Bikes  1898   
3            6   Mountain-100 Black- 48    Bikes  Mountain Bikes  1898   
4            7  Mountain-100 Silver- 38    Bikes  Mountain Bikes  1912   

  last_sale_date  recency_in_months product_segment  lifespan  total_orders  \
0     2011-12-27                161  High-Performer        11            49   
1     2011-12-27                161  High-Performer        11            45   
2     2011-12-21                161  High-Performer        11            60   
3     2011-12-26                161  High-Performer        12            57   
4     2011-12-22                161  High-Performer        12            58   

   total_sales  total_quantity  total_cus